# Fitting GLMs in Julia

One of the oldest techniques in statistics is fitting _linear models_, which encompass various regression models and the models underlying the _analysis of variance_.  In the mid-20'th century the practice of fitting a response by least squares was extended to provide for models like _logistic regression_, for a binary response or for binomial response data, and _Poisson regression_, when the response is a count.

In 1972 Nelder and Wedderburn published a paper ["Generalized Linear Models"](http://en.wikipedia.org/wiki/Generalized_linear_model) in J. Royal Statistical Society, Series A that provided a unifying framework for these and other models.  In these models a response, which can be binary or integer-valued or real-valued, is measured on $n$ occasions along with the values of _covariates_, which are also called "predictor" variables.

## Components of the model

The model is a description of the probability distribution of the response given values of parameters.  From the probability model and the observed data, both the response and the covariates, we estimate values of parameters.

The values of the covariates and the model form determine a _model matrix_, $\bf X$.  A _linear predictor_ is of the form $\bf\eta=X\beta$, where $\bf\beta$ is the _coefficient vector_ to be estimated.  In some cases there is an additional _scale parameter_ to be estimated.

### Linear regression

The probability model for linear regression is
$$ \mathcal{Y}\sim\mathcal{N}(X\beta,\rm\sigma^2\bf I)$$

In this case, the _mean response_ is the linear predictor.  That is, $$\rm E[\mathcal{Y}]=\bf\mu=\eta=\bf X\beta$$
and the distribution of the response is multivariate normal (or Gaussian) with scale parameter, $\sigma$.  Furthermore, the responses are independent.

### A link function

One characteristic of a linear predictor is that its range is a complete linear subspace of the $n$-dimensional response space.  For a normal distribution there are no constraints on the expected response, which makes it meaningful for us to set $\bf\mu=\eta$.  For other distributions the range of the expected response is constrained.  For a Bernoulli distribution we have $0\le\mu_i\le1,i=1,\dots,n$ and for a Poisson we have $0<\mu_i$.

To accomodate these constraints we map the linear predictor values, $\eta_i$, to the expected responses, $\mu_i$, in such a way that the constraints hold.  For historical reasons it is the inverse map, $g:\mu_i\rightarrow\eta_i$, that is called the _link function_.  In practice we use the inverse link function, $g^{-1}:\eta_i\rightarrow\mu_i$ more than we use the link.

### Canonical link functions

If all we require in a link function is that the range of the inverse link is the appropriate interval for the expected value, i.e. $[0,1]$ for Bernoulli responses and $(0,\infty)$ for Poisson responses, then we have a very wide range of functions to choose from.  However, Nelder and Wedderburn showed that a _canonical link function_ can be derived from the probability mass function or probability density function for distributions that are in the [exponential family](http://en.wikipedia.org/wiki/Exponential_family).  The derivation is not particularly difficult but we won't include it here.

The most common examples of canonical link functions are:
- the _logit_ or "log odds" link for the Bernoulli distribution
$$g(\mu)=\log\left(\frac{\mu}{1-\mu}\right),\quad0<\mu<1$$
- the _log_ link for the Poisson distribution
$$g(\mu)=\log(\mu),\quad0<\mu$$

Interestingly, [logistic regression](http://en.wikipedia.org/wiki/Logistic_regression) is named after the inverse link function
$$g^{-1}(\eta)=\frac{1}{1+\exp(-\eta)}$$
which is called the logistic function.

### Relating the variance to the mean

The normal distribution has many, many interesting properties, one of which is that the location (mean) and spread (standard deviation) can be controlled separately.  This is not true for most distributions.  For many distributions like the Bernoulli or the Poisson the variance or standard deviation is a function of the mean.

## Specification of a GLM

To specify a generalized linear model we must provide
- the observed responses, $\bf y$
- the model matrix, $\bf X$, derived from the values of the covariates
- the link function
- the distribution type of the response

Unlike linear least squares there is no direct method for evaluating parameter estimates, such as the maximum likelihood estimates (mle's), in a GLM.  We must use iterative methods.  Below we describe the _iteratively reweighted least squares_ (IRLS) algorithm for determining the mle's.

## Defining Julia types for GLMs

The implementation in the [GLM package](https://github.com/JuliaStats/GLM.jl) is patterned after that in [R](http://www.r-project.org).  A model is defined by a formula, a data frame in which to evaluate the identifiers in the formula, a distribution name and the name of the link, which can be omitted when using the canonical link.

Concrete types that specialize the abstract `Link` type should define methods for the scalar link function, the scalar inverse link function, and the derivative, $\frac{d\mu}{d\eta}$ as a function of $\eta$.

In [1]:
using Distributions
abstract Link

type CauchitLink <: Link end
type CloglogLink  <: Link end
type IdentityLink <: Link end
type InverseLink  <: Link end
type LogitLink <: Link end
type LogLink <: Link end
type ProbitLink <: Link end
type SqrtLink <: Link end

two(x::FloatingPoint) = one(x) + one(x)
half(x::FloatingPoint) = inv(two(x))

link(::Type{CauchitLink},μ::FloatingPoint) = tan(pi*(μ - half(μ)))
linkinv(::Type{CauchitLink},η::FloatingPoint) = half(η) + atan(η)/π
dμdη(::Type{CauchitLink},η::FloatingPoint) = inv(π*(one(η) + abs2(η)))

link(::Type{CloglogLink},μ::FloatingPoint) = log(-log(one(μ) - μ))
linkinv(::Type{CloglogLink},η::FloatingPoint) = -expm1(-exp(η))
dμdη(::Type{CloglogLink},η::FloatingPoint) = exp(η)*exp(-exp(η))

link(::Type{IdentityLink},μ::FloatingPoint) = μ
linkinv(::Type{IdentityLink},η::FloatingPoint) = η
dμdη(::Type{IdentityLink},μ::FloatingPoint) = one(μ)

link(::Type{InverseLink},μ::FloatingPoint) = inv(μ)
linkinv(::Type{InverseLink},η::FloatingPoint) = inv(η)
dμdη(::Type{InverseLink},η::FloatingPoint) = -inv(abs2(η))

link(::Type{LogitLink},μ::FloatingPoint) = log(μ/(one(μ)-μ))
linkinv(::Type{LogitLink},η::FloatingPoint) = inv(one(η) + exp(-η))
dμdη(::Type{LogitLink},η::FloatingPoint) = (e = exp(-abs(η)); e/abs2(one(e)+e))

link(::Type{LogLink},μ::FloatingPoint) = log(μ)
linkinv(::Type{LogLink},η::FloatingPoint) = exp(η)
dμdη(::Type{LogLink},η::FloatingPoint) = exp(η)

link(::Type{ProbitLink},μ::FloatingPoint) = sqrt2*erfinv((two(μ)*μ - one(μ)))
linkinv(::Type{ProbitLink},η::FloatingPoint) = (one(η) + erf(η/sqrt2))/two(η)
dμdη(::Type{ProbitLink},η::FloatingPoint) = exp(-abs2(η)/two(η))/sqrt2π

link(::Type{SqrtLink},μ::FloatingPoint) = sqrt(μ)
linkinv(::Type{SqrtLink},η::FloatingPoint) = abs2(η)
dμdη(::Type{SqrtLink},η::FloatingPoint) = η + η


dμdη (generic function with 8 methods)

__Question__: Is it guilding the lily to define functions `two` and `half` instead of using, say, `convert(typeof(μ),0.5)`?  There isn't really a problem with the representation of these values being truncated in that, for example, `big(0.5)` is exact.

In [1]:
big(0.5)

5.000000000000000000000000000000000000000000000000000000000000000000000000000000e-01

__Question:__ The methods that depend on the Link or the Distribution take the type as the first argument.  I am assuming this will make for effective dispatch if `GLMmodel` is a templated type that includes the distribution and the link as template types.

We restrict the argument to floating point types so that the log's, square roots, etc. retain the same type as the argument.  The math constants `sqrt2` and `sqrt2π` are defined in the [Distributions package](https://github.com/JuliaStats/Distributions.jl).

## Contributions from the probability distribution

The probability distribution family defines the variance as a function of the mean, $\mu$, and also defines the _deviance residuals_.

The _deviance_ is defined here as negative twice the log-likelihood setting any scale parameters to unity.  In other words, the deviance depends only on the mean vector, $\bf\mu$, which is determined by the linear predictor, $\bf\eta$, which is determined by the coefficient vector, $\bf\beta$, and the model matrix, $\bf X$.

We also define the variance assuming that any scale parameters are set to one.  It is convenient to define methods for `Base.var` that take the distribution type as the first parameter.

In [3]:
Base.var(::Type{Bernoulli},μ::FloatingPoint) = μ * (one(μ) - μ)
Base.var(::Type{Binomial},μ::FloatingPoint) =  μ * (one(μ) - μ)
Base.var(::Type{Gamma},μ::FloatingPoint) = abs2(μ)
Base.var(::Type{Normal},μ::FloatingPoint) = one(μ)
Base.var(::Type{Poisson},μ::FloatingPoint) = μ

var (generic function with 65 methods)

### A note about the Binomial distribution.

Generally we consider the response from a Binomial distribution to be the number of successes, $x$, in a fixed number, say $k$, of independent, identical Bernoulli trials.  For the purposes of GLM's we consider the response as the proportion of successes, $x/k$, and use $k$ as the _prior weights_ on the observations. It turns out that a lot of the messy code in implementations of GLMs is there to handle this one specific, but common, case.

### Deviance residuals

In [R](http://www.r-project.org) there is some confusion about what the _deviance residuals_ are.  For a Gaussian distribution the deviance, up to a constant, is the sum of squared residuals.  The _residuals_ are $y_i-\mu_i,i=1,\dots,n$.  If we just want to evaluate the deviance then we could use the squared deviance residuals.  For other distributions, it is easier to evaluate the squared deviance residuals, which are the contributions to the deviance from each observation.  (Because the observations are independent, the log-likelihood can always be written as the sum of contributions for each obserations.)

To avoid this confusion we call the function that returns the squared deviance residual for an observation, `devresid2`.

In R the function that returns the squared deviance residuals is called `devresid`.  If you apply the extractor `resid` or `residuals` with `type="deviance"` you get the signed square root of these squared deviance residuals, leading to some confusion.  (The sign is that of $y_i-\mu_i$.)

In [4]:
xlogy{T<:FloatingPoint}(x::T, y::T) = x > zero(T) ? x * log(y) : zero(T)

function devresid2{T<:FloatingPoint}(::Type{Bernoulli},y::T,μ::T,wt::T)
    omy = one(T) - y
    two(y)*wt*(xlogy(y,y/μ) + xlogy(omy,omy/(one(T)-μ)))
end
function devresid2{T<:FloatingPoint}(::Type{Binomial},y::T,μ::T,wt::T)
    devresid2(Bernoulli,y,μ,wt)
end
function devresid2{T<:FloatingPoint}(::Type{Gamma},y::T,μ::T,wt::T)
    -two(y)*wt*(log(y/μ)-(y-μ)/μ)
end
function devresid2{T<:FloatingPoint}(::Type{Normal},y::T,μ::T,wt::T)
    wt * abs2(y-μ)
end
function devresid2{T<:FloatingPoint}(::Type{Poisson},y::T,μ::T,wt::T)
    two(y)*wt*(xlogy(y,y/μ) - (y-μ))
end

devresid2 (generic function with 5 methods)

### Canonical link functions

In [5]:
canonicallink(::Type{Bernoulli}) = LogitLink
canonicallink(::Type{Binomial}) = LogitLink
canonicallink(::Type{Gamma}) = InverseLink
canonicallink(::Type{Normal}) = IdentityLink
canonicallink(::Type{Poisson}) = LogLink

canonicallink (generic function with 5 methods)

### Starting values for μ

In any iterative algorithm we must determine starting values for some part of the state of the iterations.  In this case it is easiest to determine starting values for $\mu$ from which starting values for $\eta$ can be determined using the link function.  When doing this we must avoid the boundary values for $\mu$, which the link function usually maps to $\pm\infty$ on the scale of the linear predictor.

In [6]:
mustart{T<:FloatingPoint}(::Type{Bernoulli},y::T,wt::T) = (wt*y + half(y))/(wt + one(y))
mustart{T<:FloatingPoint}(::Type{Binomial},y::T,wt::T) = mustart(Bernoulli,y,wt)
mustart{T<:FloatingPoint}(::Type{Gamma},y::T,::T) = y
mustart{T<:FloatingPoint}(::Type{Normal},y::T,::T) = y
mustart{T<:FloatingPoint}(::Type{Poisson},y::T,::T) = y + inv(convert(typeof(y),10.))

mustart (generic function with 5 methods)

## Iteratively reweighted least squares (IRLS)

The IRLS algorithm determines $\hat\beta$, the mle of the coefficient vector by iterating a process of
1. Given the current $\beta$, determine $\eta$ and $\mu$.
2. Evaluate the variance (up to a scale factor) from $\mu$ (function `var`)
3. Evaluate case weights from the prior weights, the variance and the derivative, $\frac{d\mu}{d\eta}$ and solve a weighted least squares problem.

There are two approaches to the third step - a fixed-point algorithm and an algorithm using a increment.  For both algorithms the _working residual_ is

In [7]:
wrkresid{T<:FloatingPoint}(y::T,μ::T,μη::T) = (y-μ)/μη

wrkresid (generic function with 1 method)

For the fixed-point algorithm we create a _working response_ that includes an optional _offset_, `o`, which is occasionally used in a model definition.  In most cases the offset is zero and the prior weights are unity. 

In [8]:
wrkresp{T<:FloatingPoint}(wrd::T,η::T,o::T) = wrd + η - o

wrkresp (generic function with 1 method)

The _working weights_ are defined as a function of the prior weight, `w`, the derivative, `μη` and the variance, `v`.

In [9]:
wrkwt{T<:FloatingPoint}(wt::T,μη::T,v::T) = wt*abs2(μη/v)

wrkwt (generic function with 1 method)

## A `GLMmodel` type

At first glance a reasonable design of a user-defined type for GLMs would include vectors `y`, `η`, `η`$,\dots$  However, I could not make this work conveniently if the members of the type are declared as `SharedVector`.  To make it work in general each of these shared vectors would need to have the same set of `pids` associated with it, which would generally be the case but it is not the type of thing you would want to assume without checking.  To avoid this complexity I incorporate all these vectors into a, possibly shared, matrix.  Each column corresponds to an observation so that evaluation (and distribution of the calculations to processes) is performed down columns.

In [10]:
type GLMmodel{T<:FloatingPoint,D<:Distribution,L<:Link}
    X::DenseMatrix{T}        # model matrix
    vv::DenseMatrix{T}       # rows are y,o,wt,μ,η,μη,dr,wrsd,wrsp,wwt
    β::DenseVector{T}        # coefficient vector
    δβ::DenseVector{T}       # increment
end